In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from data_manager import *
from metadata import *
from distributed_utilities import *

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_anchor.txt
File does not exist: 

In [2]:
exclude_nodes = [33]

# One stack

In [3]:
classifier_id = 30

In [4]:
stack = 'MD591'
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [26]:
t = time.time()
sys.stderr.write('running svm classifier ...')

run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', 'apply_classifiers_v3.py'),
                'stack': stack,
                'setting': classifier_setting},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
#                      kwargs_list=dict(sections=range(160,170)),
                exclude_nodes=exclude_nodes,
                argument_type='partition')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 220 seconds

running svm classifier ...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 20.565487 seconds


In [ ]:
# t = time.time()
# sys.stderr.write('interpolating scoremaps ...')

# run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d' % \
#                 {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', 'interpolate_scoremaps_v3.py'),
#                 'stack': stack,
#                 'setting': classifier_setting},
#                 kwargs_list=dict(sections=range(first_sec, last_sec+1)),
#                 exclude_nodes=exclude_nodes,
#                 argument_type='partition')

# sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 50s/section

In [6]:
t = time.time()
sys.stderr.write('visualize scoremaps ...')

add_annotation = False
downscale = 32

run_distributed4(command='%(script_path)s %(stack)s %(setting)d -b %%(first_sec)d -e %%(last_sec)d -d %(downscale)d %(add_annotation)s' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', 'visualize_scoremaps_v3.py'),
                'stack': stack,
                 'setting': classifier_setting,
                'add_annotation': '-a' if add_annotation else '',
                'downscale': downscale},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                exclude_nodes=exclude_nodes,
                argument_type='partition')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 800 seconds / stack

visualize scoremaps ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 749.581318 seconds


In [10]:
t = time.time()
sys.stderr.write('constructing score volumes ...')

run_distributed4(command='%(script_path)s %(stack)s %%(structure)s %(setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct', 'construct_score_volume_v3.py'),
                'stack': stack,
                'setting': classifier_setting},
                kwargs_list=dict(structure=all_known_structures),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 200 seconds

constructing score volumes ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 198.704855 seconds


In [12]:
t = time.time()
sys.stderr.write('compute score volume gradients...')

run_distributed4(command='%(script_path)s %(stack)s %%(structure)s %(setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct', 'compute_score_volume_gradients_v3.py'),
                'stack': stack,
                'setting': classifier_setting},
                kwargs_list=dict(structure=all_known_structures),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 70 seconds

compute score volume gradients...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 47.282430 seconds


# All nissl stacks

In [6]:
classifier_setting = 2

In [9]:
# for stack in all_nissl_stacks:
for stack in ['MD602']:
    
    first_sec, last_sec = metadata_cache['section_limits'][stack]
    
    t = time.time()
    sys.stderr.write('visualize scoremaps ...')

    add_annotation = False

    run_distributed4(command='%(script_path)s %(stack)s %(setting)d -b %%(first_sec)d -e %%(last_sec)d %(add_annotation)s' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', 'visualize_scoremaps_v3.py'),
                    'stack': stack,
                     'setting': classifier_setting,
                    'add_annotation': '-a' if add_annotation else ''},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 800 seconds / stack
            
#     t = time.time()
#     sys.stderr.write('constructing score volumes ...')

#     run_distributed4(command='%(script_path)s %(stack)s %%(structure)s %(setting)d' % \
#                     {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct', 'construct_score_volume_v3.py'),
#                     'stack': stack,
#                     'setting': classifier_setting},
#                     kwargs_list=dict(structure=all_known_structures),
#                     exclude_nodes=exclude_nodes,
#                     argument_type='single')

#     sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 200 seconds
    
#     t = time.time()
#     sys.stderr.write('compute score volume gradients...')

#     run_distributed4(command='%(script_path)s %(stack)s %%(structure)s %(setting)d' % \
#                     {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct', 'compute_score_volume_gradients_v3.py'),
#                     'stack': stack,
#                     'setting': classifier_setting},
#                     kwargs_list=dict(structure=all_known_structures),
#                     exclude_nodes=exclude_nodes,
#                     argument_type='single')

#     sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 70 seconds

visualize scoremaps ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 551.913828 seconds


# From Patch Features to Score Volumes

In [ ]:
# for stack in all_stacks:
for stack in ['MD603']:
        
    first_sec, last_sec = metadata_cache['section_limits'][stack]

    #################################

    t = time.time()
    sys.stderr.write('running svm classifier ...')


    run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', 'apply_classifiers_v3.py'),
                    'stack': stack,
                    'setting': classifier_setting},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 220 seconds

    #################################

    t = time.time()
    sys.stderr.write('interpolating scoremaps ...')

    run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', 'interpolate_scoremaps_v3.py'),
                    'stack': stack,
                    'setting': classifier_setting},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 50s/section

    #################################

    t = time.time()
    sys.stderr.write('visualize scoremaps ...')

    add_annotation = False

    run_distributed4(command='%(script_path)s %(stack)s %(setting)d -b %%(first_sec)d -e %%(last_sec)d %(add_annotation)s' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', 'visualize_scoremaps_v3.py'),
                    'stack': stack,
                     'setting': classifier_setting,
                    'add_annotation': '-a' if add_annotation else ''},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 800 seconds / stack

    #################################

    t = time.time()
    sys.stderr.write('constructing score volumes ...')

    run_distributed4(command='%(script_path)s %(stack)s %%(structure)s %(setting)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct', 'construct_score_volume_v3.py'),
                    'stack': stack,
                    'setting': classifier_setting},
                    kwargs_list=dict(structure=all_known_structures),
                    exclude_nodes=exclude_nodes,
                    argument_type='single')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 200 seconds

    #################################

    t = time.time()
    sys.stderr.write('compute score volume gradients...')

    run_distributed4(command='%(script_path)s %(stack)s %%(structure)s %(setting)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct', 'compute_score_volume_gradients_v3.py'),
                    'stack': stack,
                    'setting': classifier_setting},
                    kwargs_list=dict(structure=all_known_structures),
                    exclude_nodes=exclude_nodes,
                    argument_type='single')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t))

    
    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

# Global align

In [6]:
classifier_setting = 2
warp_setting = 1

In [6]:
# Align

t = time.time()
sys.stderr.write('align all subjects to atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'global_registration_v3.py'),
                'warp_setting': warp_setting,
                'clf_setting': classifier_setting},
#                 kwargs_list=dict(stack=all_nissl_stacks),
                 kwargs_list=dict(stack=['MD642']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t))  # 1000 seconds ~ 20 mins

align all subjects to atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 553.521348 seconds


In [6]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'transform_brains_v3_global.py'),
                'warp_setting': warp_setting,
                'clf_setting': classifier_setting},
#                 kwargs_list=dict(stack=all_nissl_stacks),
                 kwargs_list=dict(stack=['MD642']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 150 seconds, 300 for with surrounds.

transform atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 273.461052 seconds


In [7]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'visualize_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=all_nissl_stacks),
                 kwargs_list=dict(stack=['MD642']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 625 seconds

visualize aligned atlas overlay ...

['gcn-20-33.sdsc.edu'] are excluded
gcn-20-31.sdsc.edu 


KeyboardInterrupt



# Local align

In [13]:
classifier_setting = 2
warp_setting = 6

In [ ]:
# Local align

t = time.time()
sys.stderr.write('LOCAL align ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'local_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                 kwargs_list=dict(stack=['MD642']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 1300-1600 seconds

LOCAL align ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


In [ ]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'transform_brains_v3_local.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                kwargs_list=dict(stack=['MD642']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

In [ ]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'visualize_registration_v3.py'),
                'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                 kwargs_list=dict(stack=['MD642']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 190 seconds

# Transform locally transformed volumes back to atlas space

In [16]:
classifier_setting = 2
warp_setting = 3

In [17]:
t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'transform_brains_v3_reverse_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
                 kwargs_list=dict(stack=['MD603']),
#                 kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 200 seconds ~ 4 mins

transform atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 158.031314 seconds


# Measure Global Tx Confidence

In [4]:
warp_setting = 1

In [5]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_global.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD594']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 650 seconds ~ 10 mins

Measure confidence ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 696.112392 seconds


# Measure Local Tx Confidence

In [5]:
warp_setting = 5
classifier_setting = 2

In [6]:
t = time.time()
sys.stderr.write('Measure confidence ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s atlasV2 %(warp_setting)d %(clf_setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration', 'measure_confidence_v3_local.py'),
                 'clf_setting': classifier_setting,
                 'warp_setting': warp_setting},
#                  kwargs_list=dict(stack=['MD592']),
                kwargs_list=dict(stack=all_nissl_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 900 s ~ 15 mins

Measure confidence ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 1004.929929 seconds


# Snake

In [6]:
train_sample_scheme

8

In [7]:
# Graphcut

t = time.time()
sys.stderr.write('Graphcut ...')

exclude_nodes = [33]

# for stack in all_stacks:
for stack in ['MD635']:

    run_distributed4(command='%(script_path)s %(stack)s %%(name_sided)s %(train_sample_scheme)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/fit_contour_to_scoremap.py',
                    'stack': stack,
                    'train_sample_scheme': train_sample_scheme},
                    kwargs_list=dict(name_sided=[convert_to_left_name(s) for s in paired_structures] + \
                                     [convert_to_right_name(s) for s in paired_structures] + \
                                     [s for s in singular_structures]),
                    exclude_nodes=exclude_nodes,
                    argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

Graphcut ...

['gcn-20-33.sdsc.edu'] are excluded
gcn-20-31.sdsc.edu is down
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 163.411004 seconds


# Generate score volumes